## Part 2: Simple fits
Now the parametric models have been constructed and the datacard has been compiled, we are ready to start using combine for running fits. In CMS analyses we begin by blinding ourselves to the data in the signal region, and looking only at the expected results based off toys datasets (asimov or pseudo-experiments). In this exercise, we will look straight away at the observed results. Note, the python commands in this notebook are taken from `simple_fits.py`.

To run a simple best-fit for the signal strength, `r`, fixing the Higgs mass to 125 GeV, you can run the command in bash:

In [ ]:
import ROOT
from IPython.display import Image

In [ ]:
%%bash

combine -M MultiDimFit datacard_part1_with_norm.root -m 125 --freezeParameters MH --saveWorkspace -n .bestfit

# Ignore the PDF didn't factorize warning

We obtain a best-fit signal strength of `r = 1.548` i.e. the observed signal yield is 1.548 times the SM prediction.

The option `--saveWorkspace` stores a snapshot of the postfit workspace in the output file (`higgsCombine.bestfit.MultiDimFit.mH125.root`). We can load the postfit workspace and look at how the values of all the fit parameters change (compare the `clean` and `MultiDimFit` parameter snapshots):

In [ ]:
f = ROOT.TFile("higgsCombine.bestfit.MultiDimFit.mH125.root")
w = f.Get("w")
w.Print("v")

We can even plot the postfit signal-plus-background model using the workspace snapshot:

In [ ]:
n_bins = 80
binning = ROOT.RooFit.Binning(n_bins,100,180)

can = ROOT.TCanvas()
plot = w.var("CMS_hgg_mass").frame()
w.data("data_obs").plotOn( plot, binning )

# Load the S+B model
sb_model = w.pdf("model_s").getPdf("Tag0")

# Prefit
sb_model.plotOn( plot, ROOT.RooFit.LineColor(2), ROOT.RooFit.Name("prefit") )

# Postfit
w.loadSnapshot("MultiDimFit")
sb_model.plotOn( plot, ROOT.RooFit.LineColor(4), ROOT.RooFit.Name("postfit") )
r_bestfit = w.var("r").getVal()

plot.Draw()

leg = ROOT.TLegend(0.55,0.6,0.85,0.85)
leg.AddEntry("prefit", "Prefit S+B model (r=1.00)", "L")
leg.AddEntry("postfit", "Postfit S+B model (r=%.2f)"%r_bestfit, "L")
leg.Draw("Same")

can.Update()
can.Draw()

### Confidence intervals
We not only want to find the best-fit value of the signal strength, r, but also the confidence intervals. The `singles` algorithm will find the 68% CL intervals:

In [ ]:
%%bash

combine -M MultiDimFit datacard_part1_with_norm.root -m 125 --freezeParameters MH -n .singles --algo singles

To perform a likelihood scan (i.e. calculate 2NLL at fixed values of the signal strength, profiling the other parameters), we use the `grid` algorithm. We can control the number of points in the scan using the `--points` option. Also, it is important to set a suitable range for the signal strength parameter. The `singles` algorithm has shown us that the 1 stdev interval on r is around +/-0.2. 
* Use these intervals to define a suitable range for the scan, and change `lo,hi` in the following options accordingly: `--setParameterRanges r=lo,hi`.

In [ ]:
%%bash

# Change values of lo,hi to something reasonable
combine -M MultiDimFit datacard_part1_with_norm.root -m 125 --freezeParameters MH -n .scan \
--algo grid --points 20 --setParameterRanges r=lo,hi

We can use the `plot1DScan.py` function from combineTools to plot the likelihood scan:

In [ ]:
%%bash

plot1DScan.py higgsCombine.scan.MultiDimFit.mH125.root -o part2_scan

In [ ]:
# Lets open the png file and plot it here
Image(filename='part2_scan.png', width=500) 

* Do you understand what the plot is showing? What information about the signal strength parameter can be inferred from the plot?

### Extension: expected fits
To run *expected* fits we simply add `-t N` to the combine command. For `N>0`, this will generate N random toys from the model and fit each one independently. For `N=-1`, this will generate an asimov toy in which all statistical fluctuations from the model are suppressed. 

You can use the `--expectSignal 1` option to set the signal strength parameter to 1 when generating the toy. Alternatively, `--expectSignal 0` will generate a toy from the background-only model. For multiple parameter models you can set the initial values when generating the toy(s) using the `--setParameters` option of combine. For example, if you want to throw a toy where the Higgs mass is at 124 GeV and the background slope parameter `alpha` is equal to -0.05, you would add `--setParameters MH=124.0,alpha=-0.05`.

* Try running the asimov likelihood scan for `r=1` and `r=0`, and plotting them using the `plot1DScan.py` script.

In [ ]:
%%bash
# Run asimov likelihood scan for r=1

In [ ]:
%%bash
# Run asimov likelihood scan for r=0 (you may need to change the range of r)

In [ ]:
%%bash
# Plot the scans

### Extension: goodness-of-fit tests
The goodness-of-fit tests available in combine are only well-defined for binned maximum likelihood fits. Therefore, to perform a goodness-of-fit test with a parametric datacard, make sure to save the data object as a `RooDataHist`, as in `workspace_bkg_binned.root`. 

* Try editing the `datacard_part1_with_norm.txt` file to pick up the correct binned workspace file, and the `RooDataHist`. The goodness-of-fit method requires at-least one nuisance parameter in the model to run successfully. Append the following line to the end of the datacard:

```shell
lumi_13TeV      lnN          1.01         -
```

* Does the datacard compile with the `text2workspace.py` command?

In [ ]:
%%bash
# Assuming you have called the edited card: datacard_part1_binned.txt
text2workspace.py datacard_part1_binned.txt -m 125

We use the `GoodnessOfFit` method in combine to evaluate how compatible the observed data are with the model pdf. There are three types of GoF algorithm within combine, this example will use the `saturated` algorithm. You can find more information about the other algorithms [here](https://cms-analysis.github.io/HiggsAnalysis-CombinedLimit/part3/commonstatsmethods/#goodness-of-fit-tests).

Firstly, we want to calculate the value of the test statistic for the observed data:

In [ ]:
%%bash
combine -M GoodnessOfFit datacard_part1_binned.root --algo saturated -m 125 --freezeParameters MH \
-n .goodnessOfFit_data

Now lets calculate the test statistic value for many toys thrown from the model:

In [ ]:
%%bash
combine -M GoodnessOfFit datacard_part1_binned.root --algo saturated -m 125 --freezeParameters MH \
-n .goodnessOfFit_toys -t 1000

To make a plot of the GoF test-statistic distribution you can run the following commands, which first collect the values of the test-statistic into a json file, and then plots from the json file:

In [ ]:
%%bash
combineTool.py -M CollectGoodnessOfFit --input higgsCombine.goodnessOfFit_data.GoodnessOfFit.mH125.root higgsCombine.goodnessOfFit_toys.GoodnessOfFit.mH125.123456.root -m 125.0 -o gof.json

plotGof.py gof.json --statistic saturated --mass 125.0 -o part2_gof

In [ ]:
# Lets open the png file and plot it here
Image(filename='part2_gof.png', width=500) 

* What does the plot tell us? Does the model fit the data well? You can refer back to the discussion [here](https://indico.cern.ch/event/1227742/contributions/5240056/)
